Define a molecule and get integrals from a computational chemistry driver (PySCF in this case)

In [8]:
# import common packages
import numpy as np

from qiskit import Aer

# lib from Qiskit Aqua
from qiskit.aqua import QuantumInstance
from qiskit.aqua.algorithms import VQE, NumPyMinimumEigensolver
from qiskit.aqua.operators import Z2Symmetries
from qiskit.aqua.components.optimizers import COBYLA

# lib from Qiskit Aqua Chemistry
from qiskit.chemistry import FermionicOperator
from qiskit.chemistry.drivers import PySCFDriver, UnitsType
from qiskit.chemistry.components.variational_forms import UCCSD
from qiskit.chemistry.components.initial_states import HartreeFock

# Step 1: Define a molecule
Here, we use LiH in the sto3g basis with the PySCF driver as an example. The molecule object records the information from the PySCF driver.

In [9]:
# using driver to get fermionic Hamiltonian
# PySCF example
driver = PySCFDriver(atom='Li .0 .0 .0; H .0 .0 1.6', unit=UnitsType.ANGSTROM,
                     charge=0, spin=0, basis='sto3g')
molecule = driver.run()

# Step 2: Prepare qubit Hamiltonian
setup the to-be-frozen and to-be-removed orbitals to reduce the problem size when we map to the qubit Hamiltonian. Furthermore, we define the mapping type for the qubit Hamiltonian. 
For the particular parity mapping, we can further reduce the problem size.

In [10]:
# please be aware that the idx here with respective to original idx
freeze_list = [0]
remove_list = [-3, -2] # negative number denotes the reverse order
map_type = 'parity'

h1 = molecule.one_body_integrals
h2 = molecule.two_body_integrals
nuclear_repulsion_energy = molecule.nuclear_repulsion_energy

num_particles = molecule.num_alpha + molecule.num_beta
num_spin_orbitals = molecule.num_orbitals * 2
print("HF energy: {}".format(molecule.hf_energy - molecule.nuclear_repulsion_energy))
print("# of electrons: {}".format(num_particles))
print("# of spin orbitals: {}".format(num_spin_orbitals))

HF energy: -8.854072040283647
# of electrons: 4
# of spin orbitals: 12


In [11]:

# prepare full idx of freeze_list and remove_list
# convert all negative idx to positive
remove_list = [x % molecule.num_orbitals for x in remove_list]
freeze_list = [x % molecule.num_orbitals for x in freeze_list]
# update the idx in remove_list of the idx after frozen, since the idx of orbitals are changed after freezing
remove_list = [x - len(freeze_list) for x in remove_list]
remove_list += [x + molecule.num_orbitals - len(freeze_list)  for x in remove_list]
freeze_list += [x + molecule.num_orbitals for x in freeze_list]

# prepare fermionic hamiltonian with orbital freezing and eliminating, and then map to qubit hamiltonian
# and if PARITY mapping is selected, reduction qubits
energy_shift = 0.0
qubit_reduction = True if map_type == 'parity' else False

ferOp = FermionicOperator(h1=h1, h2=h2)
if len(freeze_list) > 0:
    ferOp, energy_shift = ferOp.fermion_mode_freezing(freeze_list)
    num_spin_orbitals -= len(freeze_list)
    num_particles -= len(freeze_list)
if len(remove_list) > 0:
    ferOp = ferOp.fermion_mode_elimination(remove_list)
    num_spin_orbitals -= len(remove_list)

qubitOp = ferOp.mapping(map_type=map_type, threshold=0.00000001)
qubitOp = Z2Symmetries.two_qubit_reduction(qubitOp, num_particles) if qubit_reduction else qubitOp
qubitOp.chop(10**-10)

print(qubitOp.print_details())
print(qubitOp)

IIII	(-0.20765933501970862+0j)
IIIZ	(-0.09376337484626293+0j)
IIZX	(-0.003177581454837346+0j)
IIIX	(0.003177581454837346+0j)
IIXX	(-0.00125139659997106+0j)
IIYY	(0.00125139659997106+0j)
IIZZ	(-0.2116250951511016+0j)
IIXZ	(0.019200533863110134+0j)
IIXI	(0.019200533863110134+0j)
IIZI	(0.35810269945770246+0j)
IZII	(0.09376337484626292+0j)
ZXII	(0.0031775814548373467+0j)
IXII	(0.0031775814548373467+0j)
XXII	(-0.0012513965999710547+0j)
YYII	(0.0012513965999710547+0j)
ZZII	(-0.21162509515110162+0j)
XZII	(-0.01920053386311013+0j)
XIII	(0.01920053386311013+0j)
ZIII	(-0.3581026994577025+0j)
IZIZ	(-0.12182774215821558+0j)
IZZX	(0.012144897228063112+0j)
IZIX	(-0.012144897228063112+0j)
IZXX	(0.031698745987344915+0j)
IZYY	(-0.031698745987344915+0j)
IXIZ	(0.012144897228063112+0j)
ZXIZ	(0.012144897228063112+0j)
IXZX	(-0.003265995499657571+0j)
ZXZX	(-0.003265995499657571+0j)
IXIX	(0.003265995499657571+0j)
ZXIX	(0.003265995499657571+0j)
IXXX	(-0.008650156860609206+0j)
ZXXX	(-0.008650156860609206+0j)
IX

In [5]:
# Using exact eigensolver to get the smallest eigenvalue
exact_eigensolver = NumPyMinimumEigensolver(qubitOp)
ret = exact_eigensolver.run()
print('The computed energy is: {:.12f}'.format(ret.eigenvalue.real))
print('The total ground state energy is: {:.12f}'.format(ret.eigenvalue.real + energy_shift + nuclear_repulsion_energy))

The computed energy is: -1.077059745735
The total ground state energy is: -7.881072044031


# Step 3: Initiate and configure dynamically-loaded instances
To run VQE with the UCCSD variational form, we require

VQE algorithm
Classical Optimizer
UCCSD variational form
Prepare the initial state in the HartreeFock state

In [6]:
backend = Aer.get_backend('statevector_simulator')


In [7]:
# setup COBYLA optimizer
max_eval = 200
cobyla = COBYLA(maxiter=max_eval)

# setup HartreeFock state
HF_state = HartreeFock(num_spin_orbitals, num_particles, map_type, 
                       qubit_reduction)

# setup UCCSD variational form
var_form = UCCSD(num_orbitals=num_spin_orbitals, num_particles=num_particles, 
                 active_occupied=[0], active_unoccupied=[0, 1],
                 initial_state=HF_state, qubit_mapping=map_type, 
                 two_qubit_reduction=qubit_reduction, num_time_slices=1)

# setup VQE
vqe = VQE(qubitOp, var_form, cobyla)
quantum_instance = QuantumInstance(backend=backend)